In [12]:
from openai import OpenAI

In [13]:
client = OpenAI(base_url='http://127.0.0.1:11434/v1/',
                api_key='ollama'
               )

In [14]:
from elasticsearch import Elasticsearch

In [15]:
import json

In [16]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [17]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
from tqdm.auto import tqdm

In [21]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████| 948/948 [00:04<00:00, 216.85it/s]


In [22]:
q = 'how I register the course?'

In [24]:
response = client.chat.completions.create(
    model='tinyllama',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'Sure! Here\'s a step-by-step guide on how to register for a online course at Northeastern University through our learning portal:\n\n1. Go to our learning platform, which is called Class Central. You can find it under the "Students" tab in the menu of your course.\n\n2. Once you are logged in, navigate to the Online Courses > Browse page - this will list all online courses offered by Northeastern University as well as other institutions within the DLS system.\n\n3. Find the course you are interested in and click on the "Enroll Now" button located next to its title.\n\n4. You\'ll see a progress bar indicating how many students have yet to enroll, along with some information about your upcoming enrollee(s). It should look something like:\n\n![Enrolee Progess Bar at Online Course Browse Page](/assets/images/enrprogs.png)\n\n5. You can click on that box to enroll the selected students. If all is well, the course page will refresh and you\'ll see a "Newly enrolled" message in the box under

In [55]:
def llm(prompt):
    response = client.chat.completions.create(
        model='tinyllama',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [28]:
#Q3
query = 'How do execute a command on a Kubernetes pod?'
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)
response

ObjectApiResponse({'took': 20, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'mpwJKJcB1Rv14uoyVWDm', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'KZwJKJcB1Rv14uoyV2Hj', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubern

In [42]:
#Q4
query =  "How do copy a file to a Docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [ ]:
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
What's the length of the resulting prompt? (use the len function)

In [43]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [45]:
query = "How do I execute a command in a running docker container?"
q5 = build_prompt(query, result_docs)
q5

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do I execute a command in a running docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/l

In [46]:
len(q5)

1478

In [ ]:
!pip install tiktoken

In [49]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [54]:
#Q6
len(encoding.encode(q5))

326

In [56]:
# Q7
prompt = q5
llm(prompt)

'To execute a command in a running docker container, launch the docker container with `--entrypoint bash` and attach to its shell prompt using `docker exec -it`. Also, find and mount a specific Docker container ID. Use `docker ps` to find the container-id, and then execute the commands using that ID:\n\nQ: How do I copy files from my local machine to docker container?\nA: To copy a file or directory from your local machine into a running Docker container, use the `docker cp command`. The basic syntax is as follows:\n\n```bash\n$ docker cp <source-folder> <container-id:path/in/container> /destination-folder/on/host\n```\n\nIn this example, we are copying the local folder `/home/tester/uploads` into a running container at `3ec6e0a4a17d427fcab8c4b76dafbe9a:55b4fbcad9bdeffa7b5ccb21dc0638d0a027cd57` with the destination folder `uploads`. You can modify this command depending on your specific use case. After executing the command, a symbolic link `/destination-folder/on/host` will be created

In [ ]:
# Q8

1 request: 150 input tokens, 250 output tokens
1000 requests:
Input tokens: 150 × 1000 = 150,000
Output tokens: 250 × 1000 = 250,000
Prices for gpt-4o (as of June 17):
Input: $0.005 per 1,000 tokens
Output: $0.015 per 1,000 tokens
Input cost:
150,000 tokens × $0.005 / 1,000 = $0.75

Output cost:
250,000 tokens × $0.015 / 1,000 = $3.75

Total cost:
$0.75 (input) + $3.75 (output) = $4.50